# Alphalens Analysis

### Loading libraries

In [19]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import pandas_datareader.data as web

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Model Interpreter
import shap

# Warnings
import warnings

# Path, OS & Time
import os, sys
from time import time
from pathlib import Path

# StatsModel
import statsmodels.api as sm

# Scikit-Learn
from sklearn.preprocessing import scale
from sklearn.feature_selection import mutual_info_regression

# SciPy
from scipy.stats import spearmanr

# Gradient Booster
import lightgbm as lgb

In [20]:
from utils import MultipleTimeSeriesCV

In [21]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [22]:
idx= pd.IndexSlice

sns.set_style('whitegrid')

warnings.filterwarnings('ignore')

In [23]:
results_path = Path('results')

if not results_path.exists():
    results_path.mkdir()

In [29]:
# DATA_STORE = Path('..', 'data', 'assets.h5')

### Getting Trade Prices

In [32]:
def get_trade_prices(tickers):
    return (pd.read_hdf(DATA_STORE, 'quandl/wiki/prices')
              .loc[idx['2006':'2017', tickers], 'adj_open']
              .unstack('ticker')
              .sort_index()
            .shift(-1)
            .tz_localize('UTC'))

In [ ]:
# trade_prices = get_trade_prices(tickers)

In [ ]:
# trade_prices.info()

### Loading Factors

In [ ]:
factors = (pd.concat([pd.read_hdf('data.h5', 'factors/common'),
                      pd.read_hdf('data.h5', 'factors/formulaic')
                      .rename(columns=lambda x: f'alpha_{int(x):03}')],
                     axis=1)
           .dropna(axis=1, thresh=100000)
           .sort_index())

In [ ]:
factors.info()

In [ ]:
tickers = factors.index.get_level_values('ticker').unique()

In [ ]:
alpha = 'alpha_054'